## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [3]:
cuda_use_gpus(0)

In [4]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_siamese_lstm_attention_magic'

In [6]:
RANDOM_SEED = 42

In [7]:
np.random.seed(RANDOM_SEED)

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [11]:
magic_feature_lists = [
    'magic_jturkewitz',
    'magic_stas_svd_150',
    'magic_stas_avito',
]

In [12]:
X_train_magic, X_test_magic, _ = load_feature_lists(magic_feature_lists)

In [13]:
X_train_magic = X_train_magic.values
X_test_magic = X_test_magic.values

In [14]:
scaler = StandardScaler()
scaler.fit(np.vstack([X_train_magic, X_test_magic]))
X_train_magic = scaler.transform(X_train_magic)
X_test_magic = scaler.transform(X_test_magic)

In [15]:
y_train = load(features_data_folder + 'y_train.pickle')

In [16]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [17]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [18]:
class AttentionWithContext(Layer):
    """
        Attention operation, with a context/query vector, for temporal data.
        Supports Masking.
        Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
        "Hierarchical Attention Networks for Document Classification"
        by using a context vector to assist the attention
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(AttentionWithContext())
        """

    def __init__(self, init='glorot_uniform', kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None,  **kwargs):
        self.supports_masking = True
        self.init = initializers.get(init)
        self.kernel_initializer = initializers.get('glorot_uniform')

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight((input_shape[-1], 1),
                                 initializer=self.kernel_initializer,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.kernel_regularizer,
                                 constraint=self.kernel_constraint)
        self.b = self.add_weight((input_shape[1],),
                                 initializer='zero',
                                 name='{}_b'.format(self.name),
                                 regularizer=self.bias_regularizer,
                                 constraint=self.bias_constraint)

        self.u = self.add_weight((input_shape[1],),
                                 initializer=self.kernel_initializer,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.kernel_regularizer,
                                 constraint=self.kernel_constraint)
        self.built = True

    def compute_mask(self, input, mask):
        return None

    def call(self, x, mask=None):
        # (x, 40, 300) x (300, 1)
        multData =  K.dot(x, self.kernel) # (x, 40, 1)
        multData = K.squeeze(multData, -1) # (x, 40)
        multData = multData + self.b # (x, 40) + (40,)

        multData = K.tanh(multData) # (x, 40)

        multData = multData * self.u # (x, 40) * (40, 1) => (x, 1)
        multData = K.exp(multData) # (X, 1)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            mask = K.cast(mask, K.floatx()) #(x, 40)
            multData = mask*multData #(x, 40) * (x, 40, )

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        multData /= K.cast(K.sum(multData, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        multData = K.expand_dims(multData)
        weighted_input = x * multData
        return K.sum(weighted_input, axis=1)


    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1],)

In [19]:
def create_dense_block(input_layer, num_units, dropout_rate):
    dense = Dense(num_units)(input_layer)
    bn = BatchNormalization()(dense)
    relu = Activation('relu')(bn)
    dropout = Dropout(dropout_rate)(relu)
    output = dropout
    
    return output

In [20]:
def create_model(params):
    embedding_layer = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )
    lstm_layer = LSTM(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
        return_sequences=True,
    )
    attention_layer = AttentionWithContext()

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = attention_layer(lstm_layer(embedded_sequences_1))

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = attention_layer(lstm_layer(embedded_sequences_2))

    magic_input = Input(shape=(X_train_magic.shape[-1], ))
    
    merged = concatenate([x1, y1, magic_input])
    dropout = Dropout(params['dense_dropout_rate'])(merged)
    
    dense_1 = create_dense_block(dropout, params['num_dense_1'], params['dense_dropout_rate'])
    dense_2 = create_dense_block(dropout, params['num_dense_2'], params['dense_dropout_rate'])
    
    output = Dense(1, activation='sigmoid')(dense_2)

    model = Model(
        inputs=[sequence_1_input, sequence_2_input, magic_input],
        outputs=output
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

In [21]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [22]:
NUM_FOLDS = 5
NUM_EPOCHS = 200

In [23]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [24]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [25]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS))

In [26]:
model_params = {
    'dense_dropout_rate': 0.164,
    'lstm_dropout_rate': 0.324,
    'num_dense_1': 128,
    'num_dense_2': 64,
    'num_lstm': 256,
}

In [ ]:
def predict(model, X_q1, X_q2, X_magic):
    y1 = model.predict(
        [X_q1, X_q2, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    y2 = model.predict(
        [X_q2, X_q1, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    return (y1 + y2) / 2

In [ ]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
    X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
    X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2, X_fold_train_magic], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2, X_fold_val_magic], y_fold_val),

        batch_size=64,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val] = predict(model, X_train_q1[ix_val], X_train_q2[ix_val], X_train_magic[ix_val])
    y_test_oofp[:, fold_num] = predict(model, X_test_q1, X_test_q2, X_test_magic)
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_train_magic
    del X_fold_val_q1
    del X_fold_val_q2
    del X_fold_val_magic
    del model
    gc.collect()


Fitting fold 1 of 5

Train on 646862 samples, validate on 161718 samples
Epoch 1/200
646862/646862 [==============================] - 506s - loss: 0.3784 - acc: 0.8323 - val_loss: 0.3235 - val_acc: 0.8575
Epoch 2/200
646862/646862 [==============================] - 508s - loss: 0.3447 - acc: 0.8444 - val_loss: 0.3092 - val_acc: 0.8628
Epoch 3/200
646862/646862 [==============================] - 506s - loss: 0.3303 - acc: 0.8505 - val_loss: 0.3061 - val_acc: 0.8657
Epoch 4/200
646862/646862 [==============================] - 510s - loss: 0.3216 - acc: 0.8547 - val_loss: 0.2972 - val_acc: 0.8674
Epoch 5/200
646862/646862 [==============================] - 510s - loss: 0.3160 - acc: 0.8571 - val_loss: 0.2951 - val_acc: 0.8690
Epoch 6/200
646862/646862 [==============================] - 510s - loss: 0.3114 - acc: 0.8589 - val_loss: 0.2916 - val_acc: 0.8706
Epoch 7/200
646862/646862 [==============================] - 507s - loss: 0.3074 - acc: 0.8611 - val_loss: 0.2896 - val_acc: 0.8711
Ep

646862/646862 [==============================] - 483s - loss: 0.3078 - acc: 0.8604 - val_loss: 0.2877 - val_acc: 0.8720
Epoch 7/200
646862/646862 [==============================] - 442s - loss: 0.3047 - acc: 0.8617 - val_loss: 0.2889 - val_acc: 0.8732
Epoch 8/200
646862/646862 [==============================] - 439s - loss: 0.3015 - acc: 0.8636 - val_loss: 0.2844 - val_acc: 0.8742
Epoch 9/200
646862/646862 [==============================] - 439s - loss: 0.2991 - acc: 0.8647 - val_loss: 0.2849 - val_acc: 0.8742
Epoch 10/200
646862/646862 [==============================] - 439s - loss: 0.2974 - acc: 0.8651 - val_loss: 0.2858 - val_acc: 0.8728
Epoch 11/200
646862/646862 [==============================] - 439s - loss: 0.2957 - acc: 0.8661 - val_loss: 0.2853 - val_acc: 0.8741
Epoch 12/200
646862/646862 [==============================] - 440s - loss: 0.2938 - acc: 0.8676 - val_loss: 0.2853 - val_acc: 0.8725
Epoch 00011: early stopping
2345796/2345796 [==============================] - 106s  

646866/646866 [==============================] - 440s - loss: 0.3222 - acc: 0.8540 - val_loss: 0.2978 - val_acc: 0.8675
Epoch 5/200
646866/646866 [==============================] - 439s - loss: 0.3157 - acc: 0.8574 - val_loss: 0.2978 - val_acc: 0.8684
Epoch 6/200
646866/646866 [==============================] - 439s - loss: 0.3122 - acc: 0.8584 - val_loss: 0.2927 - val_acc: 0.8692
Epoch 7/200
646866/646866 [==============================] - 440s - loss: 0.3080 - acc: 0.8605 - val_loss: 0.2930 - val_acc: 0.8694
Epoch 8/200
646866/646866 [==============================] - 440s - loss: 0.3056 - acc: 0.8615 - val_loss: 0.2887 - val_acc: 0.8716
Epoch 9/200
646866/646866 [==============================] - 440s - loss: 0.3032 - acc: 0.8629 - val_loss: 0.2913 - val_acc: 0.8684
Epoch 10/200
646866/646866 [==============================] - 439s - loss: 0.3014 - acc: 0.8639 - val_loss: 0.2873 - val_acc: 0.8726
Epoch 11/200
646866/646866 [==============================] - 439s - loss: 0.2996 - acc

646866/646866 [==============================] - 448s - loss: 0.2924 - acc: 0.8678 - val_loss: 0.2797 - val_acc: 0.8760
Epoch 15/200
646866/646866 [==============================] - 448s - loss: 0.2918 - acc: 0.8682 - val_loss: 0.2804 - val_acc: 0.8759
Epoch 00014: early stopping
2344960/2345796 [============================>.] - ETA: 0sCPU times: user 15h 3min 25s, sys: 1h 48s, total: 16h 4min 13s
Wall time: 9h 24min 36s


In [ ]:
cv_score = log_loss(y_train, y_train_oofp)
print('CV score:', cv_score)

CV score: 0.28358329936


## Save feature names

In [ ]:
feature_names = [
    'oofp_siamese_lstm_attention_magic',
]

In [ ]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [ ]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [ ]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [ ]:
y_test_oofp_mean = np.mean(y_test_oofp, axis=1).reshape((-1, 1))

In [ ]:
save(y_test_oofp_mean, features_data_folder + f'X_test_{feature_list_id}.pickle')